# Advent of Code 2022

## Day 18: Boiling Boulders

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

Day 18 was pretty easy, and only took about 20 minutes. The code was messy though, and for part 2 I used a recursion where I had to increase the Python stack size.

I re-wrote part 2 to use a deque and while loop. It doesn't really matter if we're using a stack or queue and if it's depth first or breadth first, as the problem is quite simple. Also, explicitly made a class to describe `Point3D` for a point and `Bounds3D` to calculate and check for points in a bounding cuboid. A `margin` parameter expands the box. I think this helps keep the code relatively clear and expressive. I may end up using `Point3D` again if later puzzles have 3D points in them.

I also re-wrote the `SpareLavaBlob` class to calculate the total as it goes. When a rock is placed, it's rock value is set to `6` in the rock set and total is increased by `6`. It's neighbours are incremented by `1` in the air set. When a rock is placed where it's air value is not `0`, it's value will be `6` minus air value and total is increased by `6` minus air value. When a rock is placed next to an existing rock, the rock value is decreased by `1` and the total is decreased by `1`. This keeps an invariant of the total equal to the surface area at all time, which can be read in a single operation.

### Imports

In [ ]:
from typing import Iterator
from collections import Counter, deque

### 3D Points

I used a slightly fancier `Point3D` class today than just a `namedtuple` so I could add a `neighbours` method.

In [ ]:
class Point3D:

    __slots__ = ['__tuple', '__hash', '__str', '__repr']

    def __init__(self, *, x: int, y: int, z: int):
        self.__tuple = (x, y, z)
        self.__hash = hash(self.__tuple)
        self.__repr = f'Point3D({x=}, {y=}, {z=})'
        self.__str = f'({x}, {y}, {z})'

    def __hash__(self):
        return self.__hash

    def __repr__(self):
        return self.__repr

    def __str__(self):
        return self.__str

    def __eq__(self, other):
        return type(other) == Point3D and self.__tuple == other.__tuple

    @property
    def x(self):
        return self.__tuple[0]

    @property
    def y(self):
        return self.__tuple[1]

    @property
    def z(self):
        return self.__tuple[2]

    def neighbours(self) -> list['Point3D']:
        x, y, z = self.__tuple
        return [
            Point3D(x=x-1, y=y,   z=z),
            Point3D(x=x+1, y=y,   z=z),
            Point3D(x=x,   y=y-1, z=z),
            Point3D(x=x,   y=y+1, z=z),
            Point3D(x=x,   y=y,   z=z-1),
            Point3D(x=x,   y=y,   z=z+1)
        ]

In [ ]:
class Bounds3D:

    __slots__ = ['x_min', 'x_max', 'y_min', 'y_max', 'z_min', 'z_max']

    def __init__(self, *, x_min, x_max, y_min, y_max, z_min, z_max):
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.z_min = z_min
        self.z_max = z_max

    def __contains__(self, point: Point3D) -> bool:
        if not self.x_min <= point.x <= self.x_max:
            return False
        if not self.y_min <= point.y <= self.y_max:
            return False
        if not self.z_min <= point.z <= self.z_max:
            return False
        return True

    def __str__(self):
        return f'(x∈[{self.x_min}, {self.x_max}], y∈[{self.y_min}, {self.y_max}], z∈[{self.z_min}, {self.z_max}])'

    def __repr__(self):
        return f'Bounds3D(x_min={self.x_min}, x_max={self.x_max}, y_min={self.x_min}, y_max={self.x_max}, z_min={self.z_min}, z_max={self.z_max})'

    def volume(self):
        return (self.x_max - self.x_min + 1) * (self.y_max - self.y_min + 1) * (self.z_max - self.z_min + 1)

    @staticmethod
    def containing(points: set[Point3D], margin: int = 0) -> 'Bounds3D':
        x_min = None
        x_max = None
        y_min = None
        y_max = None
        z_min = None
        z_max = None
        for p in points:
            if x_min is None or p.x < x_min:
                x_min = p.x
            if x_max is None or p.x > x_max:
                x_max = p.x
            if y_min is None or p.y < y_min:
                y_min = p.y
            if y_max is None or p.y > y_max:
                y_max = p.y
            if z_min is None or p.z < z_min:
                z_min = p.z
            if z_max is None or p.z > z_max:
                z_max = p.z
        return Bounds3D(x_min=x_min-margin,
                        x_max=x_max+margin,
                        y_min=y_min-margin,
                        y_max=y_max+margin,
                        z_min=z_min-margin,
                        z_max=z_max+margin)

### Input Parsing

In [ ]:
def read_lines(filename: str) -> Iterator[Point3D]:
    with open(filename) as file:
        for line in file:
            x, y, z = (int(i) for i in line.strip().split(','))
            yield Point3D(x=x, y=y, z=z)

### Sparse Data Structure

In [ ]:
class SparseLavaBlob:

    __slots__ = ['__rocks', '__air', '__surface_area']

    def __init__(self):
        self.__rocks: dict[Point3D, int] = {}
        self.__air: Counter[Point3D] = Counter()
        self.__surface_area = 0

    def surface_area(self) -> int:
        return self.__surface_area

    def rocks(self) -> set[Point3D]:
        return set(self.__rocks)

    def add(self, location: Point3D) -> None:

        if location in self.__rocks:
            raise ValueError(f'Duplicate rock: {location}')

        # new rock already has existing neighbours
        if location in self.__air:

            # convert from air to rock
            value = 6 - self.__air[location]
            del self.__air[location]
            self.__rocks[location] = value
            self.__surface_area += value

        # new rock has no neighbours yet
        else:

            # add new rock
            self.__rocks[location] = 6
            self.__surface_area += 6

        # update the neighbours
        for n in location.neighbours():
            if n in self.__rocks:
                self.__rocks[n] -= 1
                self.__surface_area -= 1
            elif n in self.__air:
                self.__air[n] += 1
            else:
                self.__air[n] = 1

### Pre-Processing

In [ ]:
BLOB = SparseLavaBlob()
for loc in read_lines('data/input18.txt'):
    BLOB.add(loc)

### Part 1

In [ ]:
def main():
    global BLOB
    print(f'The surface area of the lava blob is {BLOB.surface_area()} units squared.')

In [ ]:
if __name__ == '__main__':
    main()

### Part 2

In [ ]:
def flood_fill_exterior(blob: SparseLavaBlob) -> set[Point3D]:

    # get the rocks and the bounding box with margin 1
    rocks = blob.rocks()
    bound = Bounds3D.containing(rocks, margin=1)

    # setup the queue and visited nodes
    visited = set()
    queue = deque()
    queue.append(Point3D(x=bound.x_min, y=bound.y_min, z=bound.z_min))

    # iterative search using dequeue, avoid stack overflow from recursive version
    while queue:
        location = queue.pop()
        if location in bound and location not in rocks and location not in visited:
            visited.add(location)
            for n in location.neighbours():
                queue.append(n)

    # visited nodes is the set of points inside the bounding cube, but outside the rocks
    return visited

In [ ]:
def main():

    global BLOB

    # run the flood fill algorithm
    flood = flood_fill_exterior(BLOB)

    # count the neighbouring cubes of rocks which contain exterior flood
    total = sum(sum(1 for n in rock.neighbours() if n in flood) for rock in BLOB.rocks())

    print(f'The exposed exterior surface area of the lava blob is {total} units squared.')

In [ ]:
if __name__ == '__main__':
    main()